In [94]:
#load and install required packages

In [95]:
#install required packages
!pip install sklearn_crfsuite
!pip install scikit-learn 
!pip install nltk 


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [96]:
# import required libraries
import pandas as pd 
import nltk
import sklearn
import scipy.stats
import numpy as np
import seaborn as sns
import sklearn_crfsuite

#from matplotlib 
from matplotlib import pyplot as plt

#from itertools
from itertools import chain

#from sklearn
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [97]:
#load the data and ontologies

In [98]:
#this function turns the file into a list. 
def file2list(fileLocation):
    outputList = []
    with open(fileLocation, 'r', encoding='utf8') as myfile:
        sentences = myfile.read().split('\n\n')
        for sentence in sentences:
                sentenceList = []
                words = sentence.split('\n')
                for word in words:
                    wordsList = []
                    attributes = word.split(' ')
                    for attribute in attributes:
                        wordsList.append(attribute)
                    sentenceList.append(wordsList)
                outputList.append(sentenceList)
    
    return outputList

In [99]:
# train =  file2list('D:\\phd-data\\NER-annotation-data\\Dutch\\5-folds-with-pos-with-cut-sentences\\fold1.txt') 
# test =  file2list('D:\\phd-data\\NER-annotation-data\\Dutch\\5-folds-with-pos-with-cut-sentences\\fold2.txt')

#load the datasets, with training documents as train, and test documents and test
fold = "fold5"
train =  file2list('data/5-folds-train-test-split/'+ fold +'/train.txt') 
test =  file2list('data/5-folds-train-test-split/'+ fold +'/test.txt')

#remove empty line at the end of the file as this breaks the code
test.pop() 
train.pop()

[['=', 'RB', 'O'],
 ['Unidentified', 'NNP', 'O'],
 ['Quantification', 'NNP', 'O'],
 ['presented', 'VBD', 'O'],
 ['as', 'IN', 'O'],
 ['no./wt', 'NN', 'O'],
 ['(', '(', 'O'],
 ['weights', 'NNS', 'O'],
 ['in', 'IN', 'O'],
 ['grams', 'NN', 'O'],
 [')', ')', 'O'],
 [',', ',', 'O'],
 ['with', 'IN', 'O'],
 ['the', 'DT', 'O'],
 ['exception', 'NN', 'O'],
 ['of', 'IN', 'O'],
 ['amber/glass', 'NN', 'O'],
 ['beads', 'NNS', 'O'],
 ['and', 'CC', 'O'],
 ['metalwork', 'NN', 'O'],
 ['which', 'WDT', 'O'],
 ['is', 'VBZ', 'O'],
 ['presented', 'VBN', 'O'],
 ['by', 'IN', 'O'],
 ['number', 'NN', 'O'],
 ['only', 'RB', 'O'],
 ['Note', 'NNP', 'O'],
 ['-', ':', 'O'],
 ['only', 'RB', 'O'],
 ['those', 'DT', 'O'],
 ['ASGs', 'NNP', 'O'],
 ['that', 'WDT', 'O'],
 ['have', 'VBP', 'O'],
 ['produced', 'VBN', 'O'],
 ['finds', 'NNS', 'O'],
 ['are', 'VBP', 'O'],
 ['listed', 'VBN', 'O'],
 ['below', 'IN', 'O'],
 ['.', '.', 'O']]

In [100]:
%%time 
# calculates the time to open the file
        #train the NER on the list. there is one set of test and one of training. often 20:80 split
train_sent = train
test_sent = test   # tests the sent (input) of the given list as defined above
train_sent[0] # displayes the first 10 rows in the bio. - each row hs the token (effectively word), followed by pos?, and the bio label
# to identify whats below - token - label(specific label) - common derivitive of word (for posting would be post)  - then the bio label

CPU times: total: 15.6 ms
Wall time: 9.02 ms


[['INTRODUCTION', 'NNP', 'O'],
 ['7', 'CD', 'O'],
 ['1.1', 'CD', 'O'],
 ['Project', 'NN', 'O'],
 ['Background', 'NNP', 'O'],
 ['7', 'CD', 'O'],
 ['1.2', 'CD', 'O'],
 ['Site', 'NNP', 'O'],
 ['Location', 'NNP', 'O'],
 ['and', 'CC', 'O'],
 ['Description', 'NNP', 'O'],
 ['7', 'CD', 'O'],
 ['1.3', 'CD', 'O'],
 ['Archaeological', 'NNP', 'O'],
 ['Background', 'NNP', 'O'],
 ['7', 'CD', 'O'],
 ['1.4', 'CD', 'O'],
 ['Methodologies', 'NNS', 'O'],
 ['9', 'CD', 'O'],
 ['1.5', 'CD', 'O'],
 ['Professional', 'JJ', 'O'],
 ['Standards', 'NNS', 'O'],
 ['9', 'CD', 'O'],
 ['2', 'CD', 'O'],
 ['.', '.', 'O']]

In [101]:
#time to load the ontologies 

#Material ontology
materials = pd.read_csv(r'ontologies\Materials.csv') #open the file
materials[materials.columns[2]]= materials[materials.columns[2]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
materials_list = materials[materials.columns[2]].values.tolist() #turns the items into a list
print(materials_list) # outputs the list of terms to see what sort of data it contains
print("\n""\n")

#Taxon ontology
# taxon = pd.read_csv(r'ontologies\Taxon.tsv', sep="\t", error_bad_lines=False) #open the file
# taxon[taxon.columns[8]]= taxon[taxon.columns[8]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
# taxon_list = taxon[taxon.columns[8]].values.tolist() #turns the items into a list
# print(taxon_list[0:100]) # outputs the list of terms to see what sort of data it contains
# print("\n""\n")

#Animal ontology
animal = pd.read_csv(r'ontologies\VernacularName.tsv', sep="\t") #open the file
animal[animal.columns[2]]= animal[animal.columns[2]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
animal_list = animal[animal.columns[2]].values.tolist() #turns the items into a list
print(animal_list[0:100]) # outputs the list of terms to see what sort of data it contains
print("\n""\n")


#Periods ontology - for notes see above
periods = pd.read_csv(r'ontologies\Periods.csv')
periods[periods.columns[1]]= periods[periods.columns[1]].str.lower()
periods_list = periods[periods.columns[1]].values.tolist()
print(periods_list[0:100])
print("\n""\n")


#Context ontology - for notes see material 
context = pd.read_csv(r'ontologies\context.csv')
context[context.columns[0]]= context[context.columns[0]].str.lower().str.replace('*','').str.replace('<','').str.replace('>','') #replace all other symbols
context[context.columns[0]]= context[context.columns[0]].replace(to_replace =':.*',value='',regex=True) # replace the words after a colon
context_list = context[context.columns[0]].values.tolist()
print(context_list[0:100])
print("\n""\n")

['elm', 'felt', 'alabaster', 'spruce', 'tamarac', 'aluminum', 'variscite', 'argillite', 'totternhoe clunch', 'ash', 'aluminium', 'carnelian', 'cornelian', 'plaster', 'sapphire', 'paper', 'ebony', 'garnet', 'rubber', 'coal', 'emerald', 'hazel', 'puddingstone', 'hertfordshire puddingstone', 'charcoal', 'chalk', 'hydrocarbon', 'bakelite', 'amethyst', 'amphibolite', 'larch', 'siltstone', 'mudstone', 'utahlite', 'teak', 'shale', 'ivory', 'marble', 'limestone', 'leather', 'lead', 'lava', 'faience', 'jadeite', 'tooth', 'iron', 'pottery', 'greenstone', 'gold', 'glass', 'flint', 'jet', 'silver', 'pewter', 'obsidian', 'sandstone', 'object material', 'oak', 'mineral', 'wood', 'shell', 'quartz', 'slate', 'steel', 'stone', 'terracotta', 'tin', 'granite', 'quartzite', 'fir', 'antimony', 'schist', 'birch', 'lead alloy', 'zinc', 'dolerite', 'ceramic', 'pine', 'fibreglass', 'glass fibre', 'graphite', 'jade', 'onyx', 'fiberglass', 'beech', 'textile', 'metal', 'alloy', 'bronze', 'horn', 'brass', 'bone', 

C:\Users\ewout\AppData\Local\Temp\ipykernel_6192\3208070008.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  context[context.columns[0]]= context[context.columns[0]].str.lower().str.replace('*','').str.replace('<','').str.replace('>','') #replace all other symbols


In [102]:
#time to use POS

In [103]:
#this calls the function
def word2features(sent, i): 
    word = sent[i][0] #takes each token
    postag = sent[i][1] #this is the Part of Speach Tager
    
     #this tells if each token is in the ontology or not 
#     if word in materials_list: 
#         in_materials = True 
#     else:
#         in_materials = False
    
#     if word in taxon_list: 
#         in_taxon = True 
#     else:
#         in_taxon = False
    
#     if word in animal_list: 
#         in_animal = True 
#     else:
#         in_animal = False
  
#     if word in periods_list: 
#         in_periods = True 
#     else:
#         in_periods = False

#     if word in context_list: 
#         in_context = True 
#     else:
#         in_context = False
    
    #time to give each token some information     
    features = { # these are all default. 
        'bias': 1.0, # bias is just 1. 
        'word.lower()': word.lower(), # tells if the token is lower case 
        'word[-3:]': word[-3:], # takes the last four letters - the suffix
#         'Word.in_materials': in_materials, #is the token in the material ontology
#         'Word.in_taxon': in_taxon, #is the token in the material ontology
#         'Word.in_animal': in_animal, #is the token in the periods ontology 
#         'Word.in_periods': in_periods, #is the token in the periods ontology
#         'Word.in_context': in_context, #is the token in the evidence ontology
        'word.isupper()': word.isupper(), # tells if the whole token is uppercase 
        'word.istitle()': word.istitle(), # tells if the token is capital first letter
        'postag': postag,  # what is its label - Part-Of-Speech Tagger
        'postag[:2]': postag[:2],  #Takes the first three letters of the tag
        'word.isdigit()': word.isdigit(),
    }
    if i > 0: #if the token is not at the start of a sentence
        word1 = sent[i-1][0] # works out details of the token before - this is to understand the context 
        postag1 = sent[i-1][1] #what is the postag of the word before
        wordbefore = (sent[i-1][0]+ ' ' +sent[i][0]).lower #this is the token and the token before
        #if this word and word before is in the ontology then 
#         if wordbefore in materials_list: 
#             wordbefore_in_materials = True 
#         else:
#             wordbefore_in_materials = False

#         if wordbefore in taxon_list: 
#             wordbefore_in_taxon = True 
#         else:
#             wordbefore_in_taxon = False

#         if wordbefore in animal_list: 
#             wordbefore_in_animal = True 
#             print(word)
#         else:
#             wordbefore_in_animal = False

#         if wordbefore in periods_list: 
#             wordbefore_in_periods = True 
#         else:
#             wordbefore_in_periods = False

#         if wordbefore in context_list: 
#             wordbefore_in_context = True 
#         else:
#             wordbefore_in_context = False
        features.update({
            '-1:word.lower()': word1.lower(), # tells if the token is lower case
            '-1:word.istitle()': word1.istitle(), # tells if the token is capital first letter
            '-1:word.isdigit()': word1.isdigit(), # tells if the toekn is only numbers
            '-1:word.isupper()': word1.isupper(),# tells if the whole token is uppercase
#             'wordbefore_in_materials': wordbefore_in_materials,
#             'wordbefore_in_taxon': wordbefore_in_taxon,
#             'wordbefore_in_animal': wordbefore_in_animal,
#             'wordbefore_in_periods': wordbefore_in_periods,
#             'wordbefore_in_context': wordbefore_in_context,
            '-1:postag': postag1, # what was its POS tag
            '-1:postag[:2]': postag1[:2], #what is the first three POS tag of the word before
        })
    
        if i > 1: #if the token is not at the start of a sentence
                word3 = sent[i-1][0] # works out details of the token before - this is to understand the context 
                postag3 = sent[i-1][1] #what is the postag of the word before
#                 two_words_before = (sent[i-1][0]+ ' ' +sent[i-1][0]+ ' ' +sent[i][0]).lower #this is the token and the token before
                #if this word and word before is in the ontology then 
#                 if two_words_before in materials_list: 
#                     two_words_before_in_materials = True 
#                 else:
#                     two_words_before_in_materials = False

#                 if two_words_before in taxon_list: 
#                     two_words_before_in_taxon = True 
#                 else:
#                     two_words_before_in_taxon = False

#                 if two_words_before in animal_list: 
#                     two_words_before_in_animal = True 
#                     print(word)
#                 else:
#                     two_words_before_in_animal = False

#                 if two_words_before in periods_list: 
#                     two_words_before_in_periods = True 
#                 else:
#                     two_words_before_in_periods = False

#                 if two_words_before in context_list: 
#                     two_words_before_in_context = True 
#                 else:
#                     two_words_before_in_context = False
                features.update({
                    '-1:word.lower()': word1.lower(), # tells if the token is lower case
                    '-1:word.istitle()': word1.istitle(), # tells if the token is capital first letter
                    '-1:word.isdigit()': word1.isdigit(), # tells if the toekn is only numbers
                    '-1:word.isupper()': word1.isupper(),# tells if the whole token is uppercase
#                     'two_words_before_in_materials': two_words_before_in_materials,
#                     'two_words_before_in_taxon': two_words_before_in_taxon,
#                     'two_words_before_in_animal': two_words_before_in_animal,
#                     'two_words_before_in_periods': two_words_before_in_periods,
#                     'two_words_before_in_context': two_words_before_in_context,
                    '-2-postag': postag3, # what was its POS tag
                    '-2:postag[:2]': postag3[:2], #what is the first three POS tag of the word before
                })
        else:
            features['BOS2'] = True # if word is the beggining of sentence label it as so         
    else:
        features['BOS'] = True # if word is the beggining of sentence label it as so 
        
    if i < len(sent)-1: # is the word at the end of the sentence. sme as above after
        wordafter= (sent[i][0]+ ' ' +sent[i+1][0]).lower
        #this tells if the token AFTER and each token combined is in the ontology or not
#         if wordafter in materials_list: 
#             wordafter_in_materials = True 
#         else:
#             wordafter_in_materials = False

#         if wordafter in taxon_list: 
#             wordafter_in_taxon = True 
#         else:
#             wordafter_in_taxon = False

#         if wordafter in animal_list: 
#             wordafter_in_animal = True 
#         else:
#             wordafter_in_animal = False

#         if wordafter in periods_list: 
#             wordafter_in_periods = True 
#         else:
#             wordafter_in_periods = False

#         if wordafter in context_list: 
#             wordafter_in_context = True 
#         else:
#             wordafter_in_context = False
        word2 = sent[i+1][0]  
        postag2 = sent[i+1][1] 
        features.update({
#             'wordafter_in_materials': wordafter_in_materials,
#             'wordafter_in_taxon': wordafter_in_taxon,
#             'wordafter_in_animal': wordafter_in_animal,
#             'wordafter_in_periods': wordafter_in_periods,
#             'wordafter_in_context': wordafter_in_context,
            '+2:word.lower()': word2.lower(),
            '+2:word.istitle()': word2.istitle(),
            '+2:word.isupper()': word2.isupper(),
            '+2:postag': postag2,
            '+2:postag[:2]': postag2[:2],
        })
        
        if i < len(sent)-2: #if the token is not at the start of a sentence
                word4 = sent[i-1][0] # works out details of the token before - this is to understand the context 
                postag4 = sent[i-1][1] #what is the postag of the word before
#                 two_words_after = (sent[i][0]+ ' ' +sent[i+1][0]+ ' ' +sent[i+1][0]).lower#this is the token and the token before
                #if this word and word before is in the ontology then 
#                 if two_words_after in materials_list: 
#                     two_words_after_in_materials = True 
#                 else:
#                     two_words_after_in_materials = False

#                 if two_words_after in taxon_list: 
#                     two_words_after_in_taxon = True 
#                 else:
#                     two_words_after_in_taxon = False

#                 if two_words_after in animal_list: 
#                     two_words_after_in_animal = True 
#                     print(word)
#                 else:
#                     two_words_after_in_animal = False

#                 if two_words_after in periods_list: 
#                     two_words_after_in_periods = True 
#                 else:
#                    two_words_after_in_periods = False

#                 if two_words_after in context_list: 
#                     two_words_after_in_context = True 
#                 else:
#                     two_words_after_in_context = False
                features.update({
                    '-1:word.lower()': word4.lower(), # tells if the token is lower case
                    '-1:word.istitle()': word4.istitle(), # tells if the token is capital first letter
                    '-1:word.isdigit()': word4.isdigit(), # tells if the toekn is only numbers
                    '-1:word.isupper()': word4.isupper(),# tells if the whole token is uppercase
#                     'two_words_after_in_materials': two_words_after_in_materials,
#                     'two_words_after_in_taxon': two_words_after_in_taxon,
#                     'two_words_after_in_animal': two_words_after_in_animal,
#                     'two_words_after_in_periods': two_words_after_in_periods,
#                     'two_words_after_in_context': two_words_after_in_context,
                    '-2-postag': postag4, # what was its POS tag
                    '-2:postag[:2]': postag4[:2], #what is the first three POS tag of the word before
                })
        else:
            features['EOS2'] = True # if word is the beggining of sentence label it as so       
        
        
        
    else:
        features['EOS'] = True # if word is the end of sentence label it as so         
   

    if i < len(sent)-1 and i > 0: # is the word at the end of the sentence. sme as above after
            word3 = sent[i+1][0]  
            postag3 = sent[i+1][1] 
#             wordsorround = (sent[i-1][0]+ ' ' +sent[i][0]+ ' ' +sent[i+1][0]).lower
#             if wordsorround in materials_list: 
#                 wordsorround_in_materials = True 
#             else:
#                 wordsorround_in_materials = False

#             if wordsorround in taxon_list: 
#                 wordsorround_in_taxon = True 
#             else:
#                 wordsorround_in_taxon = False

#             if wordsorround in animal_list: 
#                 wordsorround_in_animal = True 
#             else:
#                 wordsorround_in_animal = False

#             if wordsorround in periods_list: 
#                 wordsorround_in_periods = True 
#             else:
#                 wordsorround_in_periods = False

#             if wordsorround in context_list: 
#                 wordsorround_in_context = True 
#             else:
#                 wordsorround_in_context = False
#             features.update({
#                 'wordsorround_in_materials': wordsorround_in_materials,
# #                 'wordsorround_in_taxon': wordsorround_in_taxon,
# #                 'wordsorround_in_animal': wordsorround_in_animal,
#                 'wordsorround_in_periods': wordsorround_in_periods,
#                 'wordsorround_in_context': wordsorround_in_context,
#             })
            if i < len(sent)-2 and i > 1: # is the word at the end of the sentence. sme as above after
                word5 = sent[i+1][0]  
                postag5 = sent[i+1][1] 
#                 twowordsorround = (sent[i-2][0]+ ' ' +sent[i-1][0]+ ' ' +sent[i][0]+ ' ' +sent[i+1][0]+ ' ' +sent[i+2][0]).lower
#                 if twowordsorround in materials_list: 
#                     twowordsorround_in_materials = True 
#                 else:
#                     twowordsorround_in_materials = False

# #                 if twowordsorround in taxon_list: 
# #                     twowordsorround_in_taxon = True 
# #                 else:
# #                     twowordsorround_in_taxon = False

#                 if twowordsorround in animal_list: 
#                     twowordsorround_in_animal = True 
#                 else:
#                     twowordsorround_in_animal = False

#                 if twowordsorround in periods_list: 
#                     twowordsorround_in_periods = True 
#                 else:
#                     twowordsorround_in_periods = False

#                 if twowordsorround in context_list: 
#                     twowordsorround_in_context = True 
#                 else:
#                     twowordsorround_in_context = False
#                 features.update({
#                     'twowordsorround_in_materials': twowordsorround_in_materials,
# #                     'twowordsorround_in_taxon': twowordsorround_in_taxon,
# #                     'twowordsorround_in_animal': twowordsorround_in_animal,
#                     'twowordsorround_in_periods': twowordsorround_in_periods,
#                     'twowordsorround_in_context': twowordsorround_in_context,                   
#                 })
            else:
                    features['OWS'] = True # if word is the end of sentence label it as so 
    else:
            features['OWS'] = True # if word is the end of sentence label it as so 
            
    return features # output these details
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))] #output for each word

def sent2labels(sent):
    return [label for token, postag, label in sent] #output for each token

def sent2tokens(sent):
    return [token for token, postag, label in sent] #output for ???


In [104]:
#print(sent2features[:2])

In [105]:
#sent2features(train_sent[0:1])[0]
[sent2features(s) for s in train_sent[0:1]]

[[{'bias': 1.0,
   'word.lower()': 'introduction',
   'word[-3:]': 'ION',
   'word.isupper()': True,
   'word.istitle()': False,
   'postag': 'NNP',
   'postag[:2]': 'NN',
   'word.isdigit()': False,
   'BOS': True,
   '+2:word.lower()': '7',
   '+2:word.istitle()': False,
   '+2:word.isupper()': False,
   '+2:postag': 'CD',
   '+2:postag[:2]': 'CD',
   '-1:word.lower()': '.',
   '-1:word.istitle()': False,
   '-1:word.isdigit()': False,
   '-1:word.isupper()': False,
   '-2-postag': '.',
   '-2:postag[:2]': '.',
   'OWS': True},
  {'bias': 1.0,
   'word.lower()': '7',
   'word[-3:]': '7',
   'word.isupper()': False,
   'word.istitle()': False,
   'postag': 'CD',
   'postag[:2]': 'CD',
   'word.isdigit()': True,
   '-1:word.lower()': 'introduction',
   '-1:word.istitle()': False,
   '-1:word.isdigit()': False,
   '-1:word.isupper()': True,
   '-1:postag': 'NNP',
   '-1:postag[:2]': 'NN',
   'BOS2': True,
   '+2:word.lower()': '1.1',
   '+2:word.istitle()': False,
   '+2:word.isupper()'

In [106]:
#time to train the NER

In [107]:
%%time
X_train = [sent2features(s) for s in train_sent] # for the token train the ner on the learned set
y_train = [sent2labels(s) for s in train_sent] # for the POS tag train the ner on the learned set

X_test = [sent2features(s) for s in test_sent] # for the token train the ner on the test set
y_test = [sent2labels(s) for s in test_sent] # for the POS tag train the ner on the test set

CPU times: total: 406 ms
Wall time: 425 ms


In [108]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.0020339643465827964,  #was initially 0.1 each
    c2=0.028003487848126302, # 'c1': 0.2963053968677204, 'c2': 0.004195898642365605
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: total: 11.8 s
Wall time: 12.1 s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.0020339643465827964,
    c2=0.028003487848126302, max_iterations=100)

In [109]:
labels = list(crf.classes_) # get the list of all labels
labels.remove('O') # remove the ones where bio is o - not got a postag
labels # show what the labels are 

['B-LOC',
 'I-LOC',
 'B-PER',
 'I-PER',
 'B-CON',
 'I-CON',
 'B-MAT',
 'B-ART',
 'I-ART',
 'I-MAT',
 'B-SPE',
 'I-SPE']

In [110]:
#now to evaluate its success

In [111]:
#calculate the f1 score
y_pred = crf.predict(X_test) # work out and predict what is the likely token
metrics.flat_f1_score(y_test, y_pred, # work out what the likely postag will be, and give it a f1 score
                      average='weighted', labels=labels)

0.5530703303964943

In [112]:
#calculate the metrics table
# group B and I results - this isnt needed, but orders the list
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
# print(metrics.classification_report(y_test, y_pred))
print(metrics.flat_classification_report(
   y_test, y_pred, labels=sorted_labels, digits=3
))
with open ("classification report "+ fold +".txt", 'w', encoding="utf-8") as f:
    f.write(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels
))

              precision    recall  f1-score   support

       B-ART      0.661     0.336     0.446       330
       I-ART      0.579     0.284     0.382       116
       B-CON      0.854     0.212     0.340       386
       I-CON      0.444     0.075     0.129        53
       B-LOC      0.841     0.670     0.746       103
       I-LOC      0.916     0.580     0.710       150
       B-MAT      0.333     0.079     0.128        38
       I-MAT      0.000     0.000     0.000         9
       B-PER      0.939     0.732     0.823       295
       I-PER      0.884     0.904     0.894       177
       B-SPE      0.824     0.389     0.528        36
       I-SPE      0.500     0.333     0.400         9

   micro avg      0.822     0.459     0.590      1702
   macro avg      0.648     0.383     0.460      1702
weighted avg      0.789     0.459     0.553      1702



In [113]:
output = "Word  actual_tag  prediction"
for x in range(len(test_sent)):
    for i in test_sent[x]:
        testWord = i[0]
        testTag = i[2]
        for j in y_pred[x]:
            prediction = j
        output += "\n" + testWord + "  " + testTag + "  " + prediction
        #print(testWord + "," + testTag + "," + prediction)
    output += "\n"
with open ("output "+ fold +".txt", 'w', encoding="utf-8") as f:
    f.write(output)
print(output)

Word  actual_tag  prediction
The  O  O
prehistoric  B-PER  O
pottery  B-ART  O
from  O  O
this  O  O
site  O  O
is  O  O
also  O  O
dated  O  O
to  O  O
the  O  O
same  O  O
period  O  O
.  O  O

Analysis  O  O
of  O  O
Potential  O  O
Even  O  O
in  O  O
association  O  O
with  O  O
the  O  O
other  O  O
loomweights  B-ART  O
from  O  O
this  O  O
site  O  O
,  O  O
there  O  O
is  O  O
limited  O  O
potential  O  O
for  O  O
further  O  O
analysis  O  O
of  O  O
these  O  O
pieces  O  O
.  O  O

WATCHING  O  O
BRIEFREPORT  O  O
Significance  O  O
of  O  O
data  O  O
The  O  O
data  O  O
has  O  O
local  O  O
significance  O  O
,  O  O
and  O  O
could  O  O
contribute  O  O
to  O  O
establishing  O  O
whether  O  O
the  O  O
form  O  O
of  O  O
loomweights  B-ART  O
in  O  O
Essex  B-LOC  O
is  O  O
related  O  O
to  O  O
a  O  O
chronology  O  O
.  O  O

Revised  O  O
Research  O  O
Aims  O  O
The  O  O
data  O  O
does  O  O
not  O  O
suggest  O  O
compelling  O  O
reasons  O  O
to  

In [45]:
#work out which of the transitions are most likely in descending order
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

#what are the 20 most likely transitions
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

#what are the 20 least likely transitions
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-CON  -> I-CON   6.616397
I-LOC  -> I-LOC   5.607146
I-CON  -> I-CON   5.108240
B-LOC  -> I-LOC   4.505791
I-ART  -> I-ART   4.160931
B-MAT  -> I-MAT   4.110137
I-PER  -> I-PER   3.914232
B-SPE  -> I-SPE   3.648257
B-ART  -> I-ART   3.598982
O      -> O       2.995678
B-PER  -> I-PER   2.750109
I-MAT  -> I-MAT   2.146045
B-SPE  -> B-ART   1.859680
I-SPE  -> I-SPE   1.790991
I-MAT  -> B-ART   1.405923
O      -> B-PER   1.379198
B-CON  -> O       1.087889
B-MAT  -> B-ART   1.038493
O      -> B-CON   0.791665
I-ART  -> B-CON   0.784281

Top unlikely transitions:
B-LOC  -> B-PER   -2.505349
O      -> I-SPE   -2.508634
B-ART  -> B-PER   -2.588106
B-PER  -> I-LOC   -2.632141
B-ART  -> B-CON   -2.933623
B-CON  -> I-ART   -3.163969
B-ART  -> I-PER   -3.331649
B-SPE  -> I-ART   -3.351932
O      -> I-MAT   -3.464353
I-ART  -> B-ART   -3.507710
I-PER  -> I-ART   -3.581651
B-LOC  -> I-PER   -3.699814
B-MAT  -> I-ART   -3.827763
B-PER  -> B-PER   -4.110701
I-PER  -> B-PER  

In [46]:
#What aspects of the terms make it likely to be that tag
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

#what are the 30 most likely aspects
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

#what 30 aspects make it least likely to be that term
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
8.239545 B-CON    word.lower():cemetery
7.312043 B-ART    word.lower():coping
6.913804 B-MAT    word.lower():cemented
6.827200 B-CON    word.lower():postholes
6.636401 B-PER    word[-3:]:xon
6.574645 B-MAT    word.lower():granite
6.542462 B-CON    word.lower():hollow-way
6.534710 I-ART    -1:word.lower():building
6.526287 B-PER    word.lower():prehistoric
6.426976 B-CON    +2:word.lower():355116
6.412253 B-ART    word.lower():mandible
6.380642 B-CON    word.lower():post-holes
6.212929 B-CON    word.lower():staircase
6.191587 B-CON    +2:word.lower():pits/
6.132275 B-CON    word.lower():roundhouses
6.122894 B-CON    word.lower():cremation
6.092030 B-ART    word.lower():chaff
6.051803 B-SPE    word.lower():barley
6.043107 B-ART    word.lower():blades
5.972890 B-CON    word.lower():granite
5.966752 O        EOS
5.959089 B-ART    word.lower():scraper
5.884596 I-PER    -1:word.lower():unspecific
5.867925 B-PER    word.lower():post-medieval
5.817248 B-MAT    word.lower():concre

In [47]:
## dont run this takes too long

In [24]:
%%time 
# this is to work out the best parameters for the testing. not needed yet but can increase the results by .1 
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
# crf = rs.best_estimator_  # shows that the best params are 
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
_x = [s.parameters['c1'] for s in rs.grid_scores_]
_y = [s.parameters['c2'] for s in rs.grid_scores_]
_c = [s.mean_validation_score for s in rs.grid_scores_]

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))